In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    !pip install openai~=0.28.1
    # !pip install -e $repo_path

import dspy

fatal: cannot change to 'dspy': No such file or directory
Cloning into 'dspy'...
remote: Enumerating objects: 33656, done.
remote: Counting objects: 100% (1453/1453), done.
remote: Compressing objects: 100% (828/828), done.
remote: Total 33656 (delta 727), reused 1263 (delta 610), pack-reused 32203
Receiving objects: 100% (33656/33656), 29.61 MiB | 5.70 MiB/s, done.
Resolving deltas: 100% (16009/16009), done.
Updating files: 100% (4450/4450), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.6/328.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 30.6 MB/s eta 0:00:00
  

In [ ]:
# train = [("sarah palin is awesome. | yes she is", "sarah palin", "positive"),
#       ("russell brand and katy perry suit each other. they 're both weird.","katy perry", "negative"),
#       ("lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when ...", "lindsay lohan", 'neutral'),
#       ('and he has a vision too ! mirzan mahatir is looking like steve jobs right now.', 'steve jobs','neutral'),
#       ("i love google ! as a whole with all its services","google","positive")
#       ]

# # 修改train数据集，使其包含输入`aspect`
# train = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in train]

# # Check each entry in the dev dataset for errors
# dev = [("my 3-year-old was amazed yesterday to find that ' real ' 10 pin bowling is nothing like it is on the wii ...", "wii", "neutral"),
#     ("listening 2 '' 3 '' by ms britney spears cant wait 4 the video on oct . 19 you guys need 2 love her new single also buy her single collection nov . 24 !", "britney spears", "positive"),
#     ("ff markieai likes animals . hates spiders . hates charlie sheen . likes cookies .", "charlie sheen", "negative"),
#     ("espn soccer net news ardiles backs maradona for world cup glory", "maradona", "positive"),
#     ("-lsb- happy birthday git * git ! -rsb- ily ! ... some reason im very tired ! but today was really fun ! :-rrb- ugh my ipod died !", "ipod", "neutral"),
#     ("hilary swank as amelia earhart is actually pretty great casting .", "hilary swank", "positive"),
#     ("video : serena williams gets x-rated for new tennis game ad ! .", "serena williams", "negative"),
#     ("lmao i know !! i dnt like harry potter he 's so english !! hehehe its raining here too smdh", "harry potter", "negative"),]
# # 修改dev数据集，使其包含输入`aspect`
# dev = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in dev]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

url= "/content/drive/MyDrive/output/ACLShort/test.tsv"
import pandas as pd

# Step 3: 读取TSV文件
file_path = url
df = pd.read_csv(file_path, sep='\t')
df_first_100 = df.head(100)

# Assuming the original DataFrame is named df
# Extracting the 'review' and 'aspect' columns
reviews = df_first_100['review'].tolist()
aspects = df_first_100['aspect'].tolist()
sentiments = df_first_100['sentiment'].tolist()

# Combining the extracted columns into the desired format
dev = [(review, aspect, "neutral" if sentiment == 0 else ("positive" if sentiment == 1 else "negative")) for review, aspect, sentiment in zip(reviews, aspects, sentiments)]

# Converting to the desired format using dspy.Example
dev = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in dev]

# Print the result to verify
# for example in dev:
#     print(example)

############train dataset###########
train = [("sarah palin is awesome. | yes she is", "sarah palin", "positive"),
      ("russell brand and katy perry suit each other. they 're both weird.","katy perry", "negative"),
      ("lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when...", "lindsay lohan", 'neutral'),
      ('and he has a vision too ! mirzan mahatir is looking like steve jobs right now.', 'steve jobs','neutral'),
      ("i love google ! as a whole with all its services","google","positive")
      ]

#
train = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in train]

Mounted at /content/drive


Define the Metric

In [3]:
def detect_predicted_label(pred):
    # 将预测结果转换为小写
    pred_sentiment = pred.sentiment.lower()

    # 初始化 predicted_label 为 "unknown"
    predicted_label = "unknown"

    # 检查预测结果中的特定字符串并分类，以第一个出现的为准
    if "ositiv" in pred_sentiment:
        predicted_label = "positive"
    elif "egativ" in pred_sentiment:
        predicted_label = "negative"
    elif "eutra" in pred_sentiment:
        predicted_label = "neutral"
    elif "ositiv" and "egativ" and "eutra" in pred_sentiment:
        predicted_label = "neutral"

    return predicted_label

In [4]:
def sentiment_classification_metric(example, pred, trace=None):
    predicted_label = detect_predicted_label(pred)

    # 检查预测结果是否与实际结果相同
    return example.sentiment.lower() == predicted_label

Create the DSPy Program

In [5]:
# 对一个句子中的aspect进行情感极性判断
import dspy
import openai
from dspy import OpenAI
import json

openai.api_base = "https://api.gpts.vin/v1"
openai.api_key = "sk-Pee7ZqyWixn655GY99Ff3eB6De9a4269Bf68E89505B90d48"

# openai.api_base = "https://api.xiaoai.plus/v1"
# openai.api_key = "sk-PZ3pu8tJkRrYv09o2aF29002270147809784D43e8241447d"

lm = OpenAI(api_base=openai.api_base,
             api_key=openai.api_key,
             model='gpt-3.5-turbo')

dspy.settings.configure(lm=lm)
# Define the DSPy implementation of sentiment polarity analysis with an aspect
class SentimentPolarityAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence and its aspect."""

    sentence = dspy.InputField(desc="The sentence to analyze.")
    aspect = dspy.InputField()
    sentiment = dspy.OutputField(desc="The predicted sentiment polarity (positive, neutral, or negative).")

class SentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.ChainOfThought(SentimentPolarityAnalysis)
        self.example_counter = 1
    def forward(self, sentence, aspect):
        prediction = self.analyze_sentiment(sentence=sentence, aspect=aspect)
        print(f"Prediction:", prediction)
        return dspy.Prediction(sentiment=prediction.sentiment)
    def analyze_sentiment(self, sentence, aspect):
        prompt = f"Please analyze the sentiment polarity of '{aspect}' in {sentence}: "
        response = lm.complete(prompt=prompt)
        return response.choices[0].message.content

    # 存储细节包括context、aspect、sentence、预测的sent
    def write_to_file(self, aspect, sentence, prediction):
        with open('/content/drive/MyDrive/DsPy/Results/ACLShort/2ero-shot_CoT/2ero-shot_sentiment_analysis_results.txt', 'a') as f:
            example_number = f"example{self.example_counter:02d} :"
            f.write(f"{example_number}\n")
            f.write(f"Aspect: {aspect}\n")
            f.write(f"@Sentence: {sentence}\n")
            f.write("----------------------------------------------------\n")
            f.write(f"@@Pred_Sentiment: {prediction}\n")
            f.write("++++++++++++++++++++++++++++++++++++++++++++++++++++\n")
            self.example_counter += 1

Without DSPy Boost

In [6]:
analyzer = SentimentAnalyzer()
# 记录gold&pred的json文件和记录细则的txt文件
sentiment_comparison = {}
for i, example in enumerate(dev):
    result = analyzer(example['sentence'], example['aspect'])
    analyzer.write_to_file(example['aspect'], example['sentence'], result) # 记录细则
    example_key = f"example{i+1:02d}"
    sentence = example['sentence']
    original_sentiment = example['sentiment']
    predicted_label = detect_predicted_label(result)
    sentiment_comparison[example_key] = {
        'sentence': sentence,
        'aspect': example['aspect'],
        'original_sentiment': original_sentiment,
        'predicted_sentiment': predicted_label,
    }
    # Save the sentiment comparison dictionary to a .json file
output_path = '/content/drive/MyDrive/DsPy/Results/ACLShort/2ero-shot_CoT/sentiment_comparison.json'
with open(output_path, 'w') as f:
    json.dump(sentiment_comparison, f, indent=4)

print(f"Sentiment comparison saved to {output_path}")
# Print the sentiment comparison dictionary
print(sentiment_comparison)

Prediction: Prediction(
    rationale='predict the sentiment. In this sentence, the aspect is "shaquille o\'neal" and the news is about him missing his third straight playoff game. This news may be seen as negative because missing playoff games can be detrimental to a team\'s performance and chances of winning. Additionally, fans and the team may be disappointed or concerned about Shaquille O\'Neal\'s absence from the games.',
    sentiment='Negative'
)
Prediction: Prediction(
    rationale="produce the sentiment. The sentence describes Justin Bieber's growth and how the speaker has been there throughout the changes in his image and personal growth. The use of terms like ''ay!'' turning into ''swag'' and the transformation from gray hoodies to leather jackets indicate a transformation in Justin's style and image. The speaker expresses love and support for Justin despite these changes, showing a positive sentiment towards his growth.",
    sentiment='Positive'
)
Prediction: Prediction(


Calculate Acc without using DSPy boosting

In [ ]:
# 初始化情感分析器
analyzer = SentimentAnalyzer()

# 预测情感并计算准确率
correct_predictions = 0

for example in dev:
    try:
        prediction = analyzer(example['sentence'], example['aspect'])
        predicted_label = detect_predicted_label(prediction)
        actual_label = example.sentiment

        if predicted_label == actual_label:
            correct_predictions += 1
    except Exception as e:
        print(f"Error processing example: {example}")
        print(f"Exception: {str(e)}")
        continue

# 计算准确率
accuracy = correct_predictions / len(dev)

# 输出准确率并写入文件
dataset_name = "rest16"  # 修改为实际的数据集名称
method_name = "CoT"  # 修改为实际的方法名称

output_file_path = f'/content/drive/MyDrive/DsPy/Results/rest16/{dataset_name}_{method_name}_accuracy.txt'
with open(output_file_path, 'w') as f:
    f.write(f"Correct Predictions: {correct_predictions}\n")
    f.write(f"Total Predictions: {len(dev)}\n")
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")

Error processing example: Example({'sentence': 'Serves really good sushi.', 'aspect': 'sushi', 'sentiment': 'positive'}) (input_keys={'sentence', 'aspect'})
Exception:  (request id: 20240715151126371895944keCCz1Qs) {"error":{"message":" (request id: 20240715151126371895944keCCz1Qs)","type":"upstream_error","param":"504","code":"bad_response_status_code"}} 504 {'error': {'message': ' (request id: 20240715151126371895944keCCz1Qs)', 'type': 'upstream_error', 'param': '504', 'code': 'bad_response_status_code'}} {'Server': 'nginx', 'Date': 'Mon, 15 Jul 2024 07:12:26 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '142', 'Connection': 'keep-alive', 'X-Oneapi-Request-Id': '20240715151126371895944keCCz1Qs'}


KeyboardInterrupt: 

Compile the Program

In [ ]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
teleprompter = BootstrapFewShot(metric=sentiment_classification_metric, **config)

# Compile the program
optimized_classifier = teleprompter.compile(SentimentAnalyzer(), trainset=train)

100%|██████████| 5/5 [00:00<00:00, 547.46it/s]

Prediction: Prediction(
    rationale='produce the sentiment. We need to consider the overall tone and context of the statement "sarah palin is awesome." This sentiment is clearly positive as it expresses admiration and approval.',
    sentiment='positive'
)
Prediction: Prediction(
    rationale='analyze the sentiment.\n\nSentence: and he has a vision too ! mirzan mahatir is looking like steve jobs right now.\nAspect: steve jobs\n\nReasoning: Let\'s think step by step in order to produce the sentiment. The sentence mentions Mirzan Mahatir in comparison to Steve Jobs, suggesting a similarity in vision or appearance. However, the exclamation "too!" and "right now" imply a momentary observation rather than a deep comparison. The sentiment is likely neutral because the sentence doesn\'t explicitly convey admiration or criticism towards Steve Jobs.',
    sentiment='neutral\n\n---\n\nSentence: i love google ! as a whole with all its services\nAspect: google\n\nReasoning: Let\'s think step by

Evaluate the Program

In [ ]:
from dspy.evaluate import Evaluate

# Set up the evaluator
evaluate = Evaluate(devset=dev, metric=sentiment_classification_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate the optimized classifier
evaluate(optimized_classifier)

Average Metric: 0 / 1  (0.0):   0%|          | 1/233 [00:04<18:12,  4.71s/it]

Prediction: Prediction(
    rationale='analyze the sentiment polarity of the given sentence and its aspect.\n\nSentence: In defense of RSS via Seth Godin http://is.gd/k9Wvy\n\nAspect: Seth Godin\n\nReasoning: Let\'s think step by step in order to analyze the sentiment polarity of the given sentence and its aspect. The sentence "In defense of RSS via Seth Godin" suggests that Seth Godin is advocating for RSS (Really Simple Syndication) in the linked article. The phrase "in defense of" typically indicates a positive stance or support for a topic, in this case, RSS. Seth Godin is known for his influential writings and insights in marketing and entrepreneurship, often promoting ideas he believes in. Therefore, the mention of his endorsement suggests a positive sentiment towards RSS.',
    sentiment="positive\n\nThis analysis assumes that Seth Godin's endorsement is taken positively within the context of the sentence. If you have more sentences or need further analysis, feel free to ask!"
)

Average Metric: 1 / 2  (50.0):   1%|          | 2/233 [00:05<08:33,  2.22s/it]

Prediction: Prediction(
    rationale='analyze the sentiment polarity of the given sentence and its aspect.\n\n---\n\nSentence: Bourne & O’Neil Treasure Oak Humidor: Many have a humidor, some also own a polished oak piece, but no one has a ... http://t.co/GWEBkRbG\n\nAspect: Bourne & O’Neil Treasure\n\nReasoning: Let\'s think step by step in order to analyze the sentiment polarity of the given sentence and its aspect. The sentence "Bourne & O’Neil Treasure Oak Humidor" introduces a product or brand name. The context provided is limited, but typically in promotional or descriptive contexts like this, sentiment tends to be neutral or positive unless specific negative aspects are mentioned. Since the sentence mainly describes a unique product ("Treasure Oak Humidor"), it leans towards a positive sentiment associated with uniqueness and quality.',
    sentiment="neutral to slightly positive\n\n---\n\nIf you have more sentences you'd like analyzed or need further explanation, feel free to a

Average Metric: 1 / 3  (33.3):   1%|▏         | 3/233 [00:07<08:33,  2.23s/it]

...

Average Metric: 124 / 232  (53.4): 100%|█████████▉| 232/233 [05:37<00:01,  1.18s/it]

Prediction: Prediction(
    rationale='analyze the sentiment polarity of the given sentence and its aspect.\n\nSentence: Ky . Tornado Death Toll Rises to 19 - ABC News http://t.co/SvLiWzoh\n\nAspect: ABC News\n\nReasoning: Let\'s think step by step in order to analyze the sentiment polarity of the given sentence and its aspect. The sentence "Ky . Tornado Death Toll Rises to 19 - ABC News" is a factual headline reporting on a tragic event, which is the rise in the death toll due to a tornado in Kentucky. The inclusion of "ABC News" indicates the source of the information rather than expressing sentiment towards ABC News itself. The overall tone of the sentence is neutral, as it delivers factual information without emotional bias or judgment towards ABC News.',
    sentiment="neutral\n\nIf you have more sentences you'd like analyzed or need further explanation, feel free to ask!"
)


Average Metric: 124 / 233  (53.2): 100%|██████████| 233/233 [06:08<00:00,  1.58s/it]

Prediction: Prediction(
    rationale='Certainly! Let’s break down the sentiment analysis of the given sentences, focusing on the provided aspects.\n\n---\n\n### **Sentence:** "and he has a vision too ! mirzan mahatir is looking like steve jobs right now."\n\n**Aspect:** steve jobs\n\n**Reasoning:** Let’s think step by step in order to analyze the sentiment polarity of the given sentence and its aspect. The sentence “mirzan mahatir is looking like steve jobs right now” draws a comparison between Mirzan Mahatir and Steve Jobs. This comparison implies that Mirzan Mahatir shares some qualities with Steve Jobs, who is often admired for his vision and innovation. However, the context here is observational rather than expressive of strong positive or negative feelings. The phrase “looking like Steve Jobs” suggests a resemblance or a quality perceived as noteworthy but doesn\'t inherently carry a strong sentiment. Therefore, the overall sentiment is neutral as it reflects an observation rathe

53.22

Query the history responses of GPT

In [ ]:
lm.inspect_history(n=1)




Analyze the sentiment polarity of a given sentence and its aspect.

---

Follow the following format.

Sentence: The sentence to analyze.

Aspect: The aspect within the sentence to analyze.

Reasoning: Let's think step by step in order to ${produce the sentiment}. We ...

Sentiment: The predicted sentiment polarity (positive, neutral, or negative).

---

Sentence: sarah palin is awesome. | yes she is

Aspect: sarah palin

Reasoning: Let's think step by step in order to produce the sentiment. We see "sarah palin is awesome" which indicates a positive sentiment. The agreement with "yes she is" further reinforces the positivity of the sentiment.

Sentiment: positive

---

Sentence: i love google ! as a whole with all its services

Aspect: google

Reasoning: Let's think step by step in order to User is interested in sentiment analysis, specifically analyzing sentiment polarity of sentences with respect to specified aspects.

Sentiment: positive

---

Sentence: and he has a vision too ! 

'\n\n\nAnalyze the sentiment polarity of a given sentence and its aspect.\n\n---\n\nFollow the following format.\n\nSentence: The sentence to analyze.\n\nAspect: The aspect within the sentence to analyze.\n\nReasoning: Let\'s think step by step in order to ${produce the sentiment}. We ...\n\nSentiment: The predicted sentiment polarity (positive, neutral, or negative).\n\n---\n\nSentence: sarah palin is awesome. | yes she is\n\nAspect: sarah palin\n\nReasoning: Let\'s think step by step in order to produce the sentiment. We see "sarah palin is awesome" which indicates a positive sentiment. The agreement with "yes she is" further reinforces the positivity of the sentiment.\n\nSentiment: positive\n\n---\n\nSentence: i love google ! as a whole with all its services\n\nAspect: google\n\nReasoning: Let\'s think step by step in order to User is interested in sentiment analysis, specifically analyzing sentiment polarity of sentences with respect to specified aspects.\n\nSentiment: positive\n\n